In [13]:
import platform 
from pathlib import Path

import numpy as np
import torch
import torchvision
import coremltools as ct
from PIL import Image

from alphapose.utils.config import update_config
from alphapose.models import builder

In [14]:
img = Image.open("test_img.png")

cfg_path = "./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml"
cfg = update_config(cfg_path)

# model_path = "./exp/delme4-256x192_res50_lr1e-3_1x.yaml/model_7.pth"
model_path = "./pretrained_models/fast_res50_256x192.pth"

model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
model.load_state_dict(torch.load(model_path), strict=False)

_IncompatibleKeys(missing_keys=['conv_out_radius.weight', 'conv_out_radius.bias', 'linear_radius.weight', 'linear_radius.bias'], unexpected_keys=[])

In [15]:
model.eval()

FastPose(
  (preact): SEResnet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)

In [16]:
# Trace with random data
random_input = torch.rand(1, 3, 256, 192)
traced_model = torch.jit.trace(model, random_input)

In [17]:
# pytorch 1.6.0
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="", shape=random_input.shape)]
)

# Set feature descriptions (these show up as comments in XCode)
model.input_description[""] = "Person image to detecto keypoints and its radius"

# Set model author name
model.author = 'Konstantin Slavnov'

# Set a short description for the Xcode UI
model.short_description = "Predicts person's keypoints and its size (aka radius)."

# Set a version for the model
model.version = "0.1"

Translating MIL ==> MLModel Ops: 100%|██████████| 782/782 [00:03<00:00, 218.15 ops/s] 


In [18]:
# Save model
export_dir = Path("export")
export_dir.mkdir(exist_ok=True)
model_path = export_dir / "fast_res50_256x192.mlmodel"
model.save(str(model_path))

# Load a saved model
loaded_model = ct.models.MLModel(str(model_path))

In [ ]:
if platform.system() == 'Darwin':
    # Make a prediction using Core ML
    # Works only on Apple system
    out = model.predict({"": img})
    print(out)

In [ ]:
# failed to work 
if platform.system() != 'Darwin':
    import tvm
    from tvm import te
    import tvm.relay as relay
    from tvm.contrib.download import download_testdata
    target = "llvm"
    
    img = np.array(img)
    x = np.transpose(img, (2, 0, 1))[np.newaxis, :]
    shape_dict = {"": x.shape}
    
    # Parse CoreML model and convert into Relay computation graph
    mod, params = relay.frontend.from_coreml(loaded_model, shape_dict)

    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target, params=params)